<a href="https://colab.research.google.com/github/anas1IA/PFA_2A/blob/main/ASL_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import numpy as np
!pip install mediapipe
import mediapipe as mp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:


# Define the directory path where your dataset is stored
dataset_dir = '/content/drive/My Drive/dataset/'

# List all classes in the dataset directory
classes = os.listdir(dataset_dir)

# Load the MediaPipe hand detection model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Function to load images from each class directory
def load_data():
    data = []
    for class_name in classes:
        class_dir = os.path.join(dataset_dir, class_name)
        images = os.listdir(class_dir)
        for img_name in images:
            img_path = os.path.join(class_dir, img_name)
            # Load image
            image = cv2.imread(img_path)
            data.append((image, class_name))
    return data



In [ ]:
# Define a function to extract features from an image using MediaPipe
def extract_features(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    if results.multi_hand_landmarks:
        # Extract hand landmarks
        hand_landmarks = results.multi_hand_landmarks[0].landmark
        # Convert landmarks to a 20-dimensional vector
        features = [landmark.x for landmark in hand_landmarks] + [landmark.y for landmark in hand_landmarks]
        return np.array(features)
    else:
        # If no hand is detected, return None
        return None




In [ ]:
# Preprocess and extract features from the dataset
def preprocess_dataset(dataset):
    processed_data = []
    for image, label in dataset:
        # Extract features from the image
        features = extract_features(image)
        if features is not None:
            # Ensure features have dimension 20
            if len(features) != 20:
                features = np.resize(features, 20)
            processed_data.append((features, label))
    return processed_data



In [ ]:
# Load dataset
dataset = load_data()

# Preprocess the dataset
processed_dataset = preprocess_dataset(dataset)

# Close the MediaPipe hands instance
hands.close()